In [ ]:
# Install required packages for Google Sheets
!pip install gspread google-auth

You can set environment variables in Colab using the `%env` magic command. Replace the placeholder values with your actual API URL, access token, and tenant ID.

In [ ]:
import requests
import os
import base64

# === Update these variables ===
url = ["OAUTH TOKEN URL"]
tenant = ["TENANT NAME"]
%env API_URL_DEV = ["API DEV URL"]
%env TENANT_ID = tenant

client_id = ["CLIENT ID"]
client_secret = ["CLIENT SECRET"]

auth_str = f"{client_id}:{client_secret}"
auth_header = "Basic " + base64.b64encode(auth_str.encode()).decode()

# === Headers ===
headers = {
    "Tenant": tenant,
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": auth_header
}

# === Data ===
data = {
    "grant_type": "client_credentials"
}

# === Make the POST request ===
response = requests.post(url, headers=headers, data=data)

# === Handle response ===
if response.status_code == 200:
    result = response.json()
    access_token = result.get("access_token")
    if access_token:
        os.environ["ACCESS_TOKEN"] = access_token
        print("✅ Access token retrieved and saved to ACCESS_TOKEN env var.")
        print("Token (first 20 chars):", access_token[:20] + "...")
    else:
        print("⚠️ Token not found in response.")
else:
    print(f"❌ Request failed with status code {response.status_code}")
    print(response.text)


env: API_URL_DEV=https://api-djarum-extension-dev.catapa.com
env: TENANT_ID=wrmk
✅ Access token retrieved and saved to ACCESS_TOKEN env var.
Token (first 20 chars): eyJ0eXAiOiJKV1QiLCJh...


After setting the environment variables, you can run the following code to make the API call:

In [ ]:
# ========================
# CONFIGURATION SECTION
# ========================

api_name = ["API NAME"]

# API Configuration
API_CONFIG = {
    # API endpoint name (will be appended to base URL)
    "api_name": api_name,  # Change this for different APIs

    # HTTP method: "POST" or "GET"
    "method": "POST",  # Change to "GET" if needed

    # Request payload for POST requests (set to None for GET)
    "payload": None,

    # URL parameters for GET requests (set to None for POST)
    "params": None,
    # Example for GET:
    # "params": {
    #     "BUKRS": "",
    #     "BUTXT": "",
    #     "limit": 100
    # }

    # Additional headers (beyond auth and tenant)
    "additional_headers": {},

    # Google Sheet naming
    "sheet_prefix": api_name,  # Prefix for the Google Sheet name
}

In [ ]:
import requests
import pandas as pd
import json
import os
from datetime import datetime

import gspread
from google.auth import default
from google.colab import auth

# Mount Google Drive and authenticate
from google.colab import drive
drive.mount('/content/drive')
auth.authenticate_user()

# ========================
# FUNCTION DEFINITIONS
# ========================

def make_api_request(api_config):
    """Make API request based on configuration"""

    # Build URL
    base_url = os.environ['API_URL_DEV']
    api_url = f"{base_url}/bapi/{api_config['api_name']}"

    # Setup headers
    headers = {
        "Authorization": f"Bearer {os.environ['ACCESS_TOKEN']}",
        "Tenant": os.environ['TENANT_ID'],
        "Content-Type": "application/json"
    }

    # Add any additional headers
    if api_config.get('additional_headers'):
        headers.update(api_config['additional_headers'])

    print(f"🌐 Making {api_config['method']} request to: {api_url}")

    # Make request based on method
    if api_config['method'].upper() == 'POST':
        if api_config.get('payload'):
            response = requests.post(api_url, headers=headers, json=api_config['payload'])
        else:
            response = requests.post(api_url, headers=headers)

    elif api_config['method'].upper() == 'GET':
        if api_config.get('params'):
            response = requests.get(api_url, headers=headers, params=api_config['params'])
        else:
            response = requests.get(api_url, headers=headers)
    else:
        raise ValueError(f"Unsupported HTTP method: {api_config['method']}")

    return response

def create_google_sheet(result_json, sheet_prefix="API_Result"):
    """Create Google Sheet from API response data"""

    try:
        # Setup Google Sheets connection
        creds, _ = default()
        gc = gspread.authorize(creds)

        # Create new Google Sheet
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        sheet_name = f"{sheet_prefix}_{timestamp}"

        print(f"📄 Creating Google Sheet: {sheet_name}")
        spreadsheet = gc.create(sheet_name)

        # Remove the default "Sheet1"
        default_sheet = spreadsheet.sheet1

        sheets_created = 0
        valid_data_found = False

        # Process each key-value pair in the response
        for key, value in result_json.items():
            if isinstance(value, list) and all(isinstance(item, dict) for item in value):
                if len(value) > 0:  # Only process non-empty lists
                    valid_data_found = True
                    df = pd.DataFrame(value)

                    # Clean sheet name (max 100 chars for Google Sheets, remove invalid chars)
                    clean_sheet_name = str(key)[:100].replace('/', '_').replace('\\', '_').replace('[', '').replace(']', '')

                    try:
                        # Create new worksheet
                        worksheet = spreadsheet.add_worksheet(
                            title=clean_sheet_name,
                            rows=max(len(df)+10, 100),
                            cols=max(len(df.columns)+5, 10)
                        )

                        # Prepare data: headers + rows
                        all_data = [df.columns.values.tolist()] + df.values.tolist()

                        # Update the worksheet with data
                        worksheet.update('A1', all_data)

                        # Format headers (bold)
                        if len(df.columns) > 0:
                            end_col = chr(65 + len(df.columns) - 1) if len(df.columns) <= 26 else f"A{len(df.columns)}"
                            worksheet.format(f'A1:{end_col}1', {
                                "textFormat": {"bold": True},
                                "backgroundColor": {"red": 0.9, "green": 0.9, "blue": 0.9}
                            })

                        sheets_created += 1
                        print(f"📊 Sheet '{clean_sheet_name}' created with {len(df)} rows and {len(df.columns)} columns")

                    except Exception as sheet_error:
                        print(f"⚠️ Error creating sheet '{clean_sheet_name}': {sheet_error}")
                        continue

        # Remove default sheet if we created other sheets
        if sheets_created > 0:
            try:
                spreadsheet.del_worksheet(default_sheet)
            except:
                pass  # Ignore if can't delete default sheet

        if valid_data_found:
            print(f"✅ Google Sheet created successfully!")
            print(f"🔗 URL: {spreadsheet.url}")
            print(f"📈 Total sheets created: {sheets_created}")

            return spreadsheet.url

        else:
            print("⚠️ No valid tabular data found in API response")
            print("Available keys in response:", list(result_json.keys()))
            return None

    except Exception as e:
        print(f"❌ Error creating Google Sheet: {e}")
        print("📋 Response structure for debugging:")
        for key, value in result_json.items():
            print(f"  - {key}: {type(value)} ({'List with ' + str(len(value)) + ' items' if isinstance(value, list) else str(value)[:50]})")
        return None

# ========================
# MAIN EXECUTION
# ========================

def run_api_to_sheets(config):
    """Main function to run API call and create Google Sheets"""

    print("🚀 Starting API to Google Sheets process...")
    print(f"📋 Configuration:")
    print(f"   - API: {config['api_name']}")
    print(f"   - Method: {config['method']}")
    print(f"   - Has Payload: {config.get('payload') is not None}")
    print(f"   - Has Params: {config.get('params') is not None}")

    try:
        # Make API request
        response = make_api_request(config)

        # Check response status
        if response.status_code in [200, 201]:
            print(f"✅ API call successful (Status: {response.status_code})")

            try:
                result_json = response.json()

                # Create Google Sheet
                sheet_url = create_google_sheet(result_json, config.get('sheet_prefix', 'API_Result'))

                if sheet_url:
                    return {
                        'success': True,
                        'sheet_url': sheet_url,
                        'status_code': response.status_code
                    }
                else:
                    return {
                        'success': False,
                        'error': 'Failed to create Google Sheet',
                        'status_code': response.status_code
                    }

            except json.JSONDecodeError:
                print(f"❌ Error: Response is not valid JSON")
                print(f"📄 Raw response: {response.text[:500]}...")
                return {
                    'success': False,
                    'error': 'Invalid JSON response',
                    'status_code': response.status_code
                }

        else:
            print(f"❌ API failed (Status: {response.status_code})")
            print(f"📄 Response: {response.text}")
            return {
                'success': False,
                'error': f'API returned status {response.status_code}',
                'response': response.text
            }

    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return {
            'success': False,
            'error': str(e)
        }

# ========================
# EXECUTE THE PROCESS
# ========================

# Run with current configuration
result = run_api_to_sheets(API_CONFIG)

if result['success']:
    print(f"\n🎉 Process completed successfully!")
    print(f"📊 Google Sheet URL: {result['sheet_url']}")
else:
    print(f"\n💥 Process failed: {result['error']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Starting API to Google Sheets process...
📋 Configuration:
   - API: Z_BPI_DOWN_ESS_MST_OBJ_OM
   - Method: POST
   - Has Payload: False
   - Has Params: False
🌐 Making POST request to: https://api-djarum-extension-dev.catapa.com/bapi/Z_BPI_DOWN_ESS_MST_OBJ_OM
✅ API call successful (Status: 201)
📄 Creating Google Sheet: Z_BPI_DOWN_ESS_MST_OBJ_OM_20250618_152315


<ipython-input-18-4215824839>:100: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', all_data)


📊 Sheet 'IT_OM' created with 31 rows and 8 columns
📊 Sheet 'IT_RELOM' created with 72 rows and 10 columns
📊 Sheet 'IT_POSDATA' created with 9 rows and 10 columns
📊 Sheet 'IT_LEADPOS' created with 19 rows and 4 columns
📊 Sheet 'IT_1003' created with 21 rows and 6 columns
📊 Sheet 'IT_POSVACANT' created with 12 rows and 5 columns
✅ Google Sheet created successfully!
🔗 URL: https://docs.google.com/spreadsheets/d/10yM8dvhri2GWlXPq8Kp05sW9CqpiME248ajtvRmxDgQ
📈 Total sheets created: 6

🎉 Process completed successfully!
📊 Google Sheet URL: https://docs.google.com/spreadsheets/d/10yM8dvhri2GWlXPq8Kp05sW9CqpiME248ajtvRmxDgQ
